# AQLクイズデータを elasticsearchに取り込む


前処理

1. AQL問題.xls をgoogle driveにアップロード
2. google spredsheetから tsv　形式でダウンロード

In [1]:
# esサーバとバージョンの足並みをそろえたほうがよいだろう。
!pip install elasticsearch==7.10.0
!pip install asyncio

You should consider upgrading via the '/home/morioka/.pyenv/versions/3.9.7/envs/deep-question-generation/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/morioka/.pyenv/versions/3.9.7/envs/deep-question-generation/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import unicodedata
import datetime

In [3]:
df = pd.read_csv('quiz-AQL-japan-2020-general.tsv', sep="\t", skiprows=1)
df.shape

(1589, 14)

In [4]:
df.head()

,ID,出題順,時事,難易度,ジャンル,問題,正解,別解・判定基準・ 解説など,裏取り,作成者,文字数,大会名,出題日,対象
0,1,1,通常問題,2.0,音楽 - J-POP,松任谷由実の楽曲『やさしさに包まれたなら』の１番の歌詞で、「やさしさに包まれたなら」と歌われ...,(静かな)木漏れ日(の),NaN,NaN,MQC,61,AQL2020全国大会一般,2021年1月10日,NaN
1,2,2,通常問題,1.0,言葉 - ことわざ,悪事を働いたにもかかわらず、かえって居直ったような態度を取ることを、何たけだけしいというでしょう？,盗人(ぬすっと/ぬすびと),NaN,NaN,MQC,49,AQL2020全国大会一般,2021年1月10日,NaN
2,3,3,通常問題,3.0,歴史 - 日本史・出来事,その調査部は「日本初のシンクタンク」とも呼ばれる、ポーツマス条約によってロシアから獲得した鉄...,南満州鉄道,NaN,NaN,MQC,61,AQL2020全国大会一般,2021年1月10日,NaN
3,4,4,通常問題,3.0,音楽 - J-POP,音楽プロジェクトの「millennium parade(ミレニアム パレード)」も主宰してい...,常田大希(つねた・だいき),NaN,NaN,MQC,92,AQL2020全国大会一般,2021年1月10日,NaN
4,5,5,通常問題,1.0,生活 - その他,今から100年前の1921年の5月5日に発売された、調香士のエルネスト・ボーが作ったシャネル...,No5,「５番」「シャネルの５番」でも〇,NaN,MQC,56,AQL2020全国大会一般,2021年1月10日,NaN


In [5]:
df.columns

Index(['ID', '出題順', '時事', '難易度', 'ジャンル', '問題', '正解', '別解・判定基準・ 解説など', '裏取り',
       '作成者', '文字数', '大会名', '出題日', '対象'],
      dtype='object')

In [6]:
df.describe()

,ID,出題順,難易度,文字数,対象
count,1589.000000,1589.000000,605.000000,1589.000000,0.0
mean,795.115167,22.212712,2.287603,62.584644,NaN
std,459.025738,12.785779,0.917054,15.476395,NaN
min,1.000000,1.000000,1.000000,24.000000,NaN
25%,398.000000,11.000000,2.000000,52.000000,NaN
50%,795.000000,22.000000,2.000000,61.000000,NaN
75%,1192.000000,33.000000,3.000000,72.000000,NaN
max,1590.000000,55.000000,5.000000,149.000000,NaN


In [7]:
# NaNだけなので除外
df = df.drop('対象', axis=1)

In [8]:
print(df.isnull().sum())

ID                  0
出題順                 0
時事                 99
難易度               984
ジャンル              984
問題                  0
正解                  0
別解・判定基準・ 解説など    1099
裏取り              1074
作成者                 0
文字数                 0
大会名                 0
出題日                 0
dtype: int64


In [9]:
# ジャンル欄をパース
df['ジャンル'] = df['ジャンル'].apply(lambda x: [y.strip() for y in str(x).split('-', 1)]).apply(lambda x: [x[0], '指定なし'] if len(x) < 2 else x).apply(lambda x: ['指定なし', x[1]] if x[0] == 'nan' else x)

In [10]:
df

,ID,出題順,時事,難易度,ジャンル,問題,正解,別解・判定基準・ 解説など,裏取り,作成者,文字数,大会名,出題日
0,1,1,通常問題,2.0,"[音楽, J-POP]",松任谷由実の楽曲『やさしさに包まれたなら』の１番の歌詞で、「やさしさに包まれたなら」と歌われ...,(静かな)木漏れ日(の),NaN,NaN,MQC,61,AQL2020全国大会一般,2021年1月10日
1,2,2,通常問題,1.0,"[言葉, ことわざ]",悪事を働いたにもかかわらず、かえって居直ったような態度を取ることを、何たけだけしいというでしょう？,盗人(ぬすっと/ぬすびと),NaN,NaN,MQC,49,AQL2020全国大会一般,2021年1月10日
2,3,3,通常問題,3.0,"[歴史, 日本史・出来事]",その調査部は「日本初のシンクタンク」とも呼ばれる、ポーツマス条約によってロシアから獲得した鉄...,南満州鉄道,NaN,NaN,MQC,61,AQL2020全国大会一般,2021年1月10日
3,4,4,通常問題,3.0,"[音楽, J-POP]",音楽プロジェクトの「millennium parade(ミレニアム パレード)」も主宰してい...,常田大希(つねた・だいき),NaN,NaN,MQC,92,AQL2020全国大会一般,2021年1月10日
4,5,5,通常問題,1.0,"[生活, その他]",今から100年前の1921年の5月5日に発売された、調香士のエルネスト・ボーが作ったシャネル...,No5,「５番」「シャネルの５番」でも〇,NaN,MQC,56,AQL2020全国大会一般,2021年1月10日
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584,1586,23,NaN,NaN,"[指定なし, 指定なし]",地震などの災害により、高層ビルの特定のフロアが平らに崩壊する現象を、ある食べ物になぞらえて「...,パンケーキクラッシュ,NaN,NaN,クイズLIVEチャンネル,61,AQL2020全国大会一般決勝,2021年2月28日
1585,1587,24,NaN,NaN,"[指定なし, 指定なし]",密度の濃い作品づくりを求め、水の密度が最大になる温度から命名されたアニメ制作会社で、『えんと...,studio４℃（よんどしい）,NaN,NaN,クイズLIVEチャンネル,78,AQL2020全国大会一般決勝,2021年2月28日
1586,1588,25,NaN,NaN,"[指定なし, 指定なし]",失敗に懲りて必要以上に用心することを、「羮（あつもの）に懲りて何を吹く」というでしょう？,膾（なます）,NaN,NaN,クイズLIVEチャンネル,44,AQL2020全国大会一般決勝,2021年2月28日
1587,1589,26,NaN,NaN,"[指定なし, 指定なし]",番号9999の小惑星にその名がつけられている、数学界の難問「フェルマーの最終定理」を証明した...,アンドリュー・ワイルズ,NaN,NaN,クイズLIVEチャンネル,61,AQL2020全国大会一般決勝,2021年2月28日


In [11]:
# 時事問題
df['時事'].value_counts()

通常問題    1345
時事問題     144
時事         1
Name: 時事, dtype: int64

In [12]:
# 表記ゆれを修正
df.loc[df[df['時事'] == '時事'].index, '時事'] = '時事問題'

In [13]:
df = df.fillna({"時事": "指定なし", "難易度": 0.0})
df = df.fillna("")

In [14]:
df

,ID,出題順,時事,難易度,ジャンル,問題,正解,別解・判定基準・ 解説など,裏取り,作成者,文字数,大会名,出題日
0,1,1,通常問題,2.0,"[音楽, J-POP]",松任谷由実の楽曲『やさしさに包まれたなら』の１番の歌詞で、「やさしさに包まれたなら」と歌われ...,(静かな)木漏れ日(の),,,MQC,61,AQL2020全国大会一般,2021年1月10日
1,2,2,通常問題,1.0,"[言葉, ことわざ]",悪事を働いたにもかかわらず、かえって居直ったような態度を取ることを、何たけだけしいというでしょう？,盗人(ぬすっと/ぬすびと),,,MQC,49,AQL2020全国大会一般,2021年1月10日
2,3,3,通常問題,3.0,"[歴史, 日本史・出来事]",その調査部は「日本初のシンクタンク」とも呼ばれる、ポーツマス条約によってロシアから獲得した鉄...,南満州鉄道,,,MQC,61,AQL2020全国大会一般,2021年1月10日
3,4,4,通常問題,3.0,"[音楽, J-POP]",音楽プロジェクトの「millennium parade(ミレニアム パレード)」も主宰してい...,常田大希(つねた・だいき),,,MQC,92,AQL2020全国大会一般,2021年1月10日
4,5,5,通常問題,1.0,"[生活, その他]",今から100年前の1921年の5月5日に発売された、調香士のエルネスト・ボーが作ったシャネル...,No5,「５番」「シャネルの５番」でも〇,,MQC,56,AQL2020全国大会一般,2021年1月10日
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584,1586,23,指定なし,0.0,"[指定なし, 指定なし]",地震などの災害により、高層ビルの特定のフロアが平らに崩壊する現象を、ある食べ物になぞらえて「...,パンケーキクラッシュ,,,クイズLIVEチャンネル,61,AQL2020全国大会一般決勝,2021年2月28日
1585,1587,24,指定なし,0.0,"[指定なし, 指定なし]",密度の濃い作品づくりを求め、水の密度が最大になる温度から命名されたアニメ制作会社で、『えんと...,studio４℃（よんどしい）,,,クイズLIVEチャンネル,78,AQL2020全国大会一般決勝,2021年2月28日
1586,1588,25,指定なし,0.0,"[指定なし, 指定なし]",失敗に懲りて必要以上に用心することを、「羮（あつもの）に懲りて何を吹く」というでしょう？,膾（なます）,,,クイズLIVEチャンネル,44,AQL2020全国大会一般決勝,2021年2月28日
1587,1589,26,指定なし,0.0,"[指定なし, 指定なし]",番号9999の小惑星にその名がつけられている、数学界の難問「フェルマーの最終定理」を証明した...,アンドリュー・ワイルズ,,,クイズLIVEチャンネル,61,AQL2020全国大会一般決勝,2021年2月28日


In [15]:
df['ジャンル1'] = df['ジャンル'].apply(lambda x: x[0])
df['ジャンル2'] = df['ジャンル'].apply(lambda x: x[1])

In [16]:
df['出題日'] = df['出題日'].apply(lambda x: datetime.datetime.strptime(x, "%Y年%m月%d日").strftime("%Y-%m-%d"))

In [17]:
df

,ID,出題順,時事,難易度,ジャンル,問題,正解,別解・判定基準・ 解説など,裏取り,作成者,文字数,大会名,出題日,ジャンル1,ジャンル2
0,1,1,通常問題,2.0,"[音楽, J-POP]",松任谷由実の楽曲『やさしさに包まれたなら』の１番の歌詞で、「やさしさに包まれたなら」と歌われ...,(静かな)木漏れ日(の),,,MQC,61,AQL2020全国大会一般,2021-01-10,音楽,J-POP
1,2,2,通常問題,1.0,"[言葉, ことわざ]",悪事を働いたにもかかわらず、かえって居直ったような態度を取ることを、何たけだけしいというでしょう？,盗人(ぬすっと/ぬすびと),,,MQC,49,AQL2020全国大会一般,2021-01-10,言葉,ことわざ
2,3,3,通常問題,3.0,"[歴史, 日本史・出来事]",その調査部は「日本初のシンクタンク」とも呼ばれる、ポーツマス条約によってロシアから獲得した鉄...,南満州鉄道,,,MQC,61,AQL2020全国大会一般,2021-01-10,歴史,日本史・出来事
3,4,4,通常問題,3.0,"[音楽, J-POP]",音楽プロジェクトの「millennium parade(ミレニアム パレード)」も主宰してい...,常田大希(つねた・だいき),,,MQC,92,AQL2020全国大会一般,2021-01-10,音楽,J-POP
4,5,5,通常問題,1.0,"[生活, その他]",今から100年前の1921年の5月5日に発売された、調香士のエルネスト・ボーが作ったシャネル...,No5,「５番」「シャネルの５番」でも〇,,MQC,56,AQL2020全国大会一般,2021-01-10,生活,その他
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584,1586,23,指定なし,0.0,"[指定なし, 指定なし]",地震などの災害により、高層ビルの特定のフロアが平らに崩壊する現象を、ある食べ物になぞらえて「...,パンケーキクラッシュ,,,クイズLIVEチャンネル,61,AQL2020全国大会一般決勝,2021-02-28,指定なし,指定なし
1585,1587,24,指定なし,0.0,"[指定なし, 指定なし]",密度の濃い作品づくりを求め、水の密度が最大になる温度から命名されたアニメ制作会社で、『えんと...,studio４℃（よんどしい）,,,クイズLIVEチャンネル,78,AQL2020全国大会一般決勝,2021-02-28,指定なし,指定なし
1586,1588,25,指定なし,0.0,"[指定なし, 指定なし]",失敗に懲りて必要以上に用心することを、「羮（あつもの）に懲りて何を吹く」というでしょう？,膾（なます）,,,クイズLIVEチャンネル,44,AQL2020全国大会一般決勝,2021-02-28,指定なし,指定なし
1587,1589,26,指定なし,0.0,"[指定なし, 指定なし]",番号9999の小惑星にその名がつけられている、数学界の難問「フェルマーの最終定理」を証明した...,アンドリュー・ワイルズ,,,クイズLIVEチャンネル,61,AQL2020全国大会一般決勝,2021-02-28,指定なし,指定なし


In [18]:
df.to_csv('quiz-AQL-japan-2020-general_clean.tsv', index=False, sep="\t")

https://qiita.com/satto_sann/items/8a63761bbfd6542bb9a2
Python Elasticsearch 基本的な使い方まとめ - Qiita

https://qiita.com/t2hk/items/a5b647f4ca764b073a47 <備忘録> Elasticsearch、Kibana、Sudachi環境を構築する - Qiita

    バルクアップデート

https://qiita.com/kasug623/items/e19c12e809254bace91b ElasticsearchにおけるMapping の使い方 - Qiita